In [1]:
import seaborn as sns
sns.set(font= "simhei",style="whitegrid",font_scale=1.6)
import matplotlib
matplotlib.rcParams['axes.unicode_minus']=False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#配置字体参数雅黑
plt.rcParams['font.sans-serif'] = 'Microsoft YaHei'
#安装字体
plt.rcParams['font.sans-serif'] = 'SimHei'
#配置矢量图
%config InlineBackend.figure_format = 'svg'

In [3]:
from statsmodels.tsa.seasonal import STL,seasonal_decompose

In [4]:
df1 = pd.read_excel("./Attachment 2.xlsx")

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56778 entries, 0 to 56777
Data columns (total 4 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   日期(年/月/日) (Date Y/M/D)                       56778 non-null  datetime64[ns]
 1   发货城市 (Delivering city)                       56778 non-null  object        
 2   收货城市 (Receiving city)                        56778 non-null  object        
 3   快递运输数量(件) (Express delivery quantity (PCS))  56778 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.7+ MB


In [6]:
df1.columns = ["日期","发货城市","收货城市","快递运输数量"]
df1 = df1[df1["快递运输数量"] > 0].reset_index(drop=True)
df1

,日期,发货城市,收货城市,快递运输数量
0,2020-04-28,R,O,216
1,2020-04-28,R,L,320
2,2020-04-28,R,G,110
3,2020-04-28,R,D,96
4,2020-04-28,P,D,144
...,...,...,...,...
53861,2023-04-27,I,S,47
53862,2023-04-27,I,E,65
53863,2023-04-27,J,I,337
53864,2023-04-27,J,K,242


In [7]:
df1["线路"] = df1["发货城市"] +"->"+ df1["收货城市"]

In [8]:
df1

,日期,发货城市,收货城市,快递运输数量,线路
0,2020-04-28,R,O,216,R->O
1,2020-04-28,R,L,320,R->L
2,2020-04-28,R,G,110,R->G
3,2020-04-28,R,D,96,R->D
4,2020-04-28,P,D,144,P->D
...,...,...,...,...,...
53861,2023-04-27,I,S,47,I->S
53862,2023-04-27,I,E,65,I->E
53863,2023-04-27,J,I,337,J->I
53864,2023-04-27,J,K,242,J->K


In [9]:
df1["年份"] = df1.日期.dt.year
df1["季度"] = df1.日期.dt.quarter

In [10]:
# 筛选出需要的线路和年份数据
df_V2N = df1[(df1["线路"] == "V->N") & (df1["年份"] == 2022)&(df1["季度"] == 3)]
df_V2Q = df1[(df1["线路"] == "V->Q") & (df1["年份"] == 2022)&(df1["季度"] == 3)]
df_J2I = df1[(df1["线路"] == "J->I") & (df1["年份"] == 2023)&(df1["季度"] == 1)]
df_O2G = df1[(df1["线路"] == "O->G") & (df1["年份"] == 2023)&(df1["季度"] == 1)]

In [11]:
df_V2N

,日期,发货城市,收货城市,快递运输数量,线路,年份,季度
36457,2022-07-01,V,N,163,V->N,2022,3
36540,2022-07-02,V,N,184,V->N,2022,3
36623,2022-07-03,V,N,170,V->N,2022,3
36708,2022-07-04,V,N,202,V->N,2022,3
36793,2022-07-05,V,N,245,V->N,2022,3
...,...,...,...,...,...,...,...
41868,2022-09-22,V,N,42,V->N,2022,3
41903,2022-09-23,V,N,42,V->N,2022,3
42051,2022-09-28,V,N,44,V->N,2022,3
42091,2022-09-29,V,N,44,V->N,2022,3


In [12]:
df_V2Q

,日期,发货城市,收货城市,快递运输数量,线路,年份,季度
36456,2022-07-01,V,Q,148,V->Q,2022,3
36539,2022-07-02,V,Q,164,V->Q,2022,3
36622,2022-07-03,V,Q,159,V->Q,2022,3
36707,2022-07-04,V,Q,142,V->Q,2022,3
36792,2022-07-05,V,Q,171,V->Q,2022,3
...,...,...,...,...,...,...,...
41477,2022-09-13,V,Q,42,V->Q,2022,3
41572,2022-09-15,V,Q,42,V->Q,2022,3
41618,2022-09-16,V,Q,42,V->Q,2022,3
41710,2022-09-18,V,Q,42,V->Q,2022,3


In [17]:
result_V2N=seasonal_decompose(df_V2N["快递运输数量"], model='addtive', period=10)
result_V2Q=seasonal_decompose(df_V2Q["快递运输数量"], model='addtive', period=7)
result_J2I=seasonal_decompose(df_J2I["快递运输数量"], model='addtive', period=10)
result_O2G=seasonal_decompose(df_O2G["快递运输数量"], model='addtive', period=7)

In [18]:
## 固定需求的值,取整
V2N_min = np.int32(result_V2N.trend.fillna(value=10000).min())
V2Q_min = np.int32(result_V2Q.trend.fillna(value=10000).min())
J2I_min = np.int32(result_J2I.trend.fillna(value=10000).min())
O2G_min = np.int32(result_O2G.trend.fillna(value=10000).min())

In [19]:
df_result = pd.DataFrame({
    '线路': ['V->N', 'V->Q', 'J->I', 'O->G'],
    '年份': [2022, 2022, 2023, 2023],
    '季度': [3, 3, 1, 1],
    '固定需求常数': [V2N_min, V2Q_min, J2I_min, O2G_min]
})

In [20]:
df_result

,线路,年份,季度,固定需求常数
0,V->N,2022,3,43
1,V->Q,2022,3,42
2,J->I,2023,1,112
3,O->G,2023,1,49
